In [1]:
# from typing import List
# from langchain.utils.openai_functions import convert_pydantic_to_openai_function
# from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
# from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
# from langchain.utilities.openapi import OpenAPISpec
import os
import openai
import param
from dotenv import load_dotenv

#Depricated
#from langchain.tools.render import format_tool_to_openai_function
from langchain_core.utils.function_calling import convert_to_openai_function

from langchain.prompts import MessagesPlaceholder
from langchain.prompts import ChatPromptTemplate

#Depricated
#from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI

from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.agent import AgentFinish


from langchain.agents import tool
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationEntityMemory

# Custom utils
from utils.scrape_method import *
from utils.link_scraper import text_summary
from utils.bot_tools import get_current_temperature, search_wikipedia, scrape_news

load_dotenv()
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
text_summary("https://www.occ.gov/publications-and-resources/publications/quarterly-report-on-bank-trading-and-derivatives-activities/files/q3-2023-derivatives-quarterly.html")

'An official website of the United States government Parts of this site may be down for maintenance from Saturday, January 27, 7:00 a.m. to Sunday, January 28, 12:00 p.m. (Eastern). An official website of theUnited States government Third Quarter 2023 Quarterly Report on Bank Trading and Derivatives Activities Share This Page:  This publication is part of: Collection:Quarterly Report on Bank Trading and Derivatives Activities Transmittal NR 2023-138, OCC Reports Third Quarter 2023 Bank Trading Revenue The Office of the Comptroller of the Currency (OCC) reported cumulative trading revenue of U.S. commercial banks and savings associations of $13.2 billion in the third quarter of 2023. The third quarter trading revenue was $470 million, or 3.4 percent, less than in the previous quarter and $474 million, or 3.7 percent, more than in the third quarter of 2022. In the report,Quarterly Report on Bank Trading and Derivatives Activities,the OCC also reported that as of the third quarter of 2023

In [2]:
### BOT 1

tools = [get_current_temperature, scrape_news]

functions = [convert_to_openai_function(f) for f in tools]
model = ChatOpenAI(temperature=0).bind(functions=functions)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

chain = prompt | model | OpenAIFunctionsAgentOutputParser()

agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory, return_intermediate_steps=True)

def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = agent_chain.invoke({
            "input": user_input, 
            "intermediate_steps": intermediate_steps
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "scrape_news": scrape_news, 
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [16]:
run_agent("scrape news")

No new OCC links found.


AgentFinish(return_values={'output': "I'm sorry, but I couldn't find any new news articles from the OCC website. Is there anything else I can assist you with?"}, log="I'm sorry, but I couldn't find any new news articles from the OCC website. Is there anything else I can assist you with?")

In [5]:
run_agent("What's the weather in SF?")

AgentFinish(return_values={'output': 'The current temperature in San Francisco is 56.0°F.'}, log='The current temperature in San Francisco is 56.0°F.')

In [3]:
### BOT 2
AGENT_CONTEXT="""
You are a sassy bot which can converse or use specific tools. 
If a question is asked which uses a tool, don't assume--ask clarifying questions where needed.
When approriate ask questions at the end of responses to keep the conversation going.
Keep answers short--only 3-4 sentences max.

If you call the scrape_news tool and it returns a link. On the next call (usually a request to summarize) to that tool you need to pass the link (i.e. everything after http:...) as an argument
"""
tools = [get_current_temperature, search_wikipedia, scrape_news]

functions = [convert_to_openai_function(f) for f in tools]
model = ChatOpenAI(temperature=0).bind(functions=functions)
prompt = ChatPromptTemplate.from_messages([
    ("system", AGENT_CONTEXT),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

chain = prompt | model | OpenAIFunctionsAgentOutputParser()

agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [ ]:
chats = agent_executor.invoke({"input": "what is the weather?"})

chats["output"]

In [ ]:
agent_executor.invoke({"input": "Denver"})

In [ ]:
agent_executor.invoke({"input": "Wait no, San Francisco"})

In [4]:

#### AGENT CLASS

class cbfs(param.Parameterized):
    
    def __init__(self, tools, **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.functions = [convert_to_openai_function(f) for f in tools]
        self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)
        self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", AGENT_CONTEXT),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        
        self.qa = AgentExecutor(
            agent=self.chain, 
            tools=tools, 
            verbose=True, 
            memory=self.memory
            )
    
    def convchain(self, query):
        if not query:
            return
        result = self.qa.invoke({"input": query})
        self.answer = result['output']
        return result

    # Define your callback functions here
    def log_tool_outputs(tool_name, tool_output):
        print(f"Tool '{tool_name}' output: {tool_output}")
    
    def print_chat_history(self):
        # Retrieve messages from memory
        history = self.memory.load_memory_variables({})
        # history will be a dictionary with 'history' key containing the conversation
        return history

    def clr_history(self,count=0):
        self.chat_history = []
        return 

In [4]:
#import langchain

#langchain.debug = True

In [5]:
cb = cbfs(tools)

cb.convchain("Turn the lights on?")



> Entering new AgentExecutor chain...
I'm sorry, but I am not capable of turning lights on or off. I'm just a sassy bot here to provide information and have a conversation. Is there anything else I can help you with?

> Finished chain.


{'input': 'Turn the lights on?',
 'chat_history': [HumanMessage(content='Turn the lights on?'),
  AIMessage(content="I'm sorry, but I am not capable of turning lights on or off. I'm just a sassy bot here to provide information and have a conversation. Is there anything else I can help you with?")],
 'output': "I'm sorry, but I am not capable of turning lights on or off. I'm just a sassy bot here to provide information and have a conversation. Is there anything else I can help you with?"}

In [6]:
cb.convchain("Are there an new OCC links?")



> Entering new AgentExecutor chain...

Invoking: `scrape_news` with `{}`


No new OCC links found.
No new OCC links found.I'm sorry, but I couldn't find any new links from OCC (Oakland Community College) at the moment. Is there anything else I can assist you with?

> Finished chain.


{'input': 'Are there an new OCC links?',
 'chat_history': [HumanMessage(content='Turn the lights on?'),
  AIMessage(content="I'm sorry, but I am not capable of turning lights on or off. I'm just a sassy bot here to provide information and have a conversation. Is there anything else I can help you with?"),
  HumanMessage(content='Are there an new OCC links?'),
  AIMessage(content="I'm sorry, but I couldn't find any new links from OCC (Oakland Community College) at the moment. Is there anything else I can assist you with?")],
 'output': "I'm sorry, but I couldn't find any new links from OCC (Oakland Community College) at the moment. Is there anything else I can assist you with?"}

In [29]:
cb.print_chat_history()

{'chat_history': [HumanMessage(content='Turn the lights on?'),
  AIMessage(content="I'm sorry, but I am not capable of turning lights on or off. I'm just a sassy bot here to chat and provide information. Is there anything else I can help you with?"),
  HumanMessage(content='Are there an new OCC links?'),
  AIMessage(content='Yes, there is a new article from the OCC titled "OCC Releases CRA Evaluations for 18 National Banks and Federal Savings Associations". You can find the article [here](https://www.occ.gov/news-issuances/news-releases/2024/nr-occ-2024-9.html). Is there anything specific you would like to know about this article?')]}

In [7]:
cb.convchain("Can you summarize the new OCC article? Pass the link to the scrape_news tool")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad> > 5:chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad> > 5:chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output"

{'input': 'Can you summarize the new OCC article? Pass the link to the scrape_news tool',
 'chat_history': [HumanMessage(content='Turn the lights on?'),
  AIMessage(content="I'm sorry, but I am not capable of turning lights on or off. I'm just a sassy bot here to provide information and have a conversation. Is there anything else I can help you with?"),
  HumanMessage(content='Are there an new OCC links?'),
  AIMessage(content='Yes, there are 2 new articles from the OCC. The titles are "Federal Bank Regulatory Agencies Seek Comment on Interagency Effort to Reduce Regulatory Burden" and "Economic Growth and Regulatory Paperwork Reduction Act of 1996: Regulatory Review to Identify Outdated, Unnecessary, or Unduly Burdensome Regulations". Here are the links to the articles: [Link 1](https://www.occ.gov/news-issuances/news-releases/2024/nr-ia-2024-10.html) and [Link 2](https://www.occ.gov/news-issuances/bulletins/2024/bulletin-2024-5.html). Let me know if you want me to summarize any of th